In [1]:
import os
import pathlib

import earthaccess
import geopandas as gpd
import hvplot.pandas
import rioxarray as rxr
import xarray as xr
import shapely

/opt/conda/envs/swot/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
earthaccess.login()

In [3]:
# Search for the SWOT dataset
swot_datasets = earthaccess.search_datasets(keyword="SWOT Discharge")
for dataset in swot_datasets:
    try:
        print(dataset['umm']['CollectionCitations'][0]['Title'], 
              dataset['umm']['ShortName'])
    except:
        print('Dataset missing title information')

SWOT Level 4 River Discharge Products Version 1 SWOT_L4_DAWG_SOS_DISCHARGE
SWOT Level 2 River Single-Pass Vector Data Product, Version C SWOT_L2_HR_RiverSP_2.0
SWOT Level 2 Water Mask Pixel Cloud Auxiliary Data Product, Version D SWOT_L2_HR_PIXCVec_D
SWOT Level 2 River Cycle-Averaged Data Product, Version C SWOT_L2_HR_RiverAvg_2.0
SWOT Level 2 River Single-Pass Vector Data Product, Version D SWOT_L2_HR_RiverSP_D
SWOT Level 2 River Single-Pass Vector Node Data Product, Version C SWOT_L2_HR_RiverSP_node_2.0
SWOT Level 2 River Single-Pass Vector Reach Data Product, Version C SWOT_L2_HR_RiverSP_reach_2.0


In [4]:
# Find data
discharge_meta = earthaccess.search_data(short_name="SWOT_L2_HR_RiverAvg_2.0")
for meta in discharge_meta:
    print(
        meta['umm']['TemporalExtent']['RangeDateTime']['BeginningDateTime'], 
        meta['umm']['TemporalExtent']['RangeDateTime']['EndingDateTime'])
    break

2023-07-21T05:33:45.768Z 2023-08-11T02:18:53.064Z


In [5]:
# Plot swaths
swaths = []
for meta in discharge_meta:
    coords = meta['umm']['SpatialExtent']['HorizontalSpatialDomain']['Geometry']['BoundingRectangles'][0]
    dates = meta['umm']['TemporalExtent']['RangeDateTime']
    bbox = shapely.geometry.box(
        minx=coords['WestBoundingCoordinate'],
        miny=coords['SouthBoundingCoordinate'],
        maxx=coords['EastBoundingCoordinate'],
        maxy=coords['NorthBoundingCoordinate'])
    swaths.append(dict(
        start=dates['BeginningDateTime'],
        end=dates['EndingDateTime'],
        geometry=bbox
    ))
swath_gdf = gpd.GeoDataFrame(swaths, geometry='geometry')
swath_gdf.hvplot(geo=True, tiles='CartoLight', hover_cols='all', fill_alpha=0)

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]   (index,start,end)

In [6]:
test_data = earthaccess.search_data(
    short_name="SWOT_L2_HR_RiverAvg_2.0",
    bounding_box=(-130, 45, -120, 50),
    temporal=('2024-01-05', '2024-01-25')
)
test_data

[Collection: {'Version': '2.0', 'ShortName': 'SWOT_L2_HR_RiverAvg_2.0'}
 Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'BoundingRectangles': [{'WestBoundingCoordinate': -136.05492053921128, 'SouthBoundingCoordinate': 42.350424960570614, 'EastBoundingCoordinate': -110.51540854839693, 'NorthBoundingCoordinate': 59.5809948519265}]}, 'Track': {'Cycle': 9, 'Passes': [{'Pass': 11}, {'Pass': 13}, {'Pass': 24}, {'Pass': 39}, {'Pass': 52}, {'Pass': 67}, {'Pass': 95}, {'Pass': 123}, {'Pass': 134}, {'Pass': 151}, {'Pass': 162}, {'Pass': 177}, {'Pass': 179}, {'Pass': 190}, {'Pass': 205}, {'Pass': 207}, {'Pass': 218}, {'Pass': 233}, {'Pass': 235}, {'Pass': 246}, {'Pass': 261}, {'Pass': 263}, {'Pass': 274}, {'Pass': 289}, {'Pass': 291}, {'Pass': 302}, {'Pass': 317}, {'Pass': 330}, {'Pass': 358}, {'Pass': 373}, {'Pass': 401}, {'Pass': 412}, {'Pass': 429}, {'Pass': 440}, {'Pass': 457}, {'Pass': 468}, {'Pass': 483}, {'Pass': 485}, {'Pass': 496}]}}}
 Temporal coverage: {'RangeDateTime': {'

In [7]:
test_ds = earthaccess.open(test_data)
test_ds

QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 1235.80it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:01<00:00,  1.61s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<00:00, 10131.17it/s]


[<File-like object HTTPFileSystem, https://archive.swot.podaac.earthdata.nasa.gov/podaac-swot-ops-cumulus-protected/SWOT_L2_HR_RiverAvg_2.0/SWOT_L2_HR_RiverAvg_009_NA_78_20240104T033426_20240125T001932_PGC0_01.zip>]

In [12]:
username = os.environ['IPLANT_USER']
download_path = pathlib.Path.home() / 'data-store' / 'data' / 'iplant' / 'home' / 'eculler' / 'swot' / 'test_data'
download_path.mkdir(exist_ok=True, parents=True)
earthaccess.download(test_data)

QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 749.65it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:03<00:00,  3.34s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<00:00, 10034.22it/s]


['/home/jovyan/data-store/swot-precip-validation/notebooks/data/2025-06-06-705018/SWOT_L2_HR_RiverAvg_009_NA_78_20240104T033426_20240125T001932_PGC0_01.zip']

In [ ]:
help(earthaccess.download)

In [11]:
dir(earthaccess)

['Auth',
 'DataCollection',
 'DataCollections',
 'DataGranule',
 'DataGranules',
 'DataServices',
 'Optional',
 'PROD',
 'Store',
 'UAT',
 '__all__',
 '__annotations__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__getattr__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__store__',
 '__version__',
 '_auth',
 '_lock',
 '_store',
 'api',
 'auth',
 'auth_environ',
 'collection_query',
 'consolidate_metadata',
 'daac',
 'dmrpp_zarr',
 'download',
 'exceptions',
 'formatters',
 'get_edl_token',
 'get_fsspec_https_session',
 'get_requests_https_session',
 'get_s3_credentials',
 'get_s3_filesystem',
 'get_s3fs_session',
 'granule_query',
 'kerchunk',
 'logger',
 'logging',
 'login',
 'open',
 'open_virtual_dataset',
 'open_virtual_mfdataset',
 'results',
 'search',
 'search_data',
 'search_datasets',
 'search_services',
 'services',
 'store',
 'system',
 'threading',
 'utils',
 'version']